In [2]:
import os, os.path
import numpy as np
import pandas as pd
import model_attributes as ma
from attribute_table import AttributeTable
import setup_analysis as sa
import support_classes as sc
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
import geopandas as gpd
import rioxarray as rx
import itertools
import model_afolu as mafl

/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_attributes.py:2635: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_attributes.py:2392: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for desig, df in df_by_designation:


In [145]:
#df_soil = pd.read_csv(sa.fp_csv_soc_fields_by_country_simple)
fields_index = ["iso_alpha_3", "year"]
df_check = df_soil[fields_index]
df_check["count"] = 1
agg = sf.simple_df_agg(
    df_check, 
    fields_index,
    {"count": "sum"}
).sort_values(by = ["count"], ascending = False)

agg[agg["count"] > 1]


/var/folders/8m/3ll2cn6d1hdcs6gjqxr2jx5d2hffc9/T/ipykernel_18255/1833089009.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_check["count"] = 1


,count,iso_alpha_3,year
5663,4,NLD,2015
5685,4,NLD,2037
5679,4,NLD,2031
5680,4,NLD,2032
5681,4,NLD,2033
5682,4,NLD,2034
5683,4,NLD,2035
5684,4,NLD,2036
5686,4,NLD,2038
5677,4,NLD,2029


In [3]:
dir_data = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/AFOLU"

# set names 
fn_climates = "kc_1984_2013.tif"
fn_countries = "WB_countries_Admin0_10m"
fn_cw = "values_info_with_cw_kc_1984_2013.csv"

fp_climates = os.path.join(dir_data, fn_climates)
fp_countries = os.path.join(dir_data, fn_countries, f"{fn_countries}.shp")
fp_cw = os.path.join(dir_data, fn_cw)

model_afolu = mafl.AFOLU(sa.model_attributes)
regions = sc.Regions(sa.model_attributes)
time_periods = sc.TimePeriods(sa.model_attributes)

/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/support_functions.py:1354: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for i, df in df_in_grouped:
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/support_functions.py:1354: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for i, df in df_in_grouped:


In [4]:
rx_array = rx.open_rasterio(fp_climates)
df_climates = rx_array[0].to_pandas()

In [5]:
df_climates.shape

(18000, 43200)

In [3]:
# convert geotiff to dataframe
rx_array = rx.open_rasterio(fp_climates)
df_climates = rx_array[0].to_pandas()
# retrieve climate categories
df_climate_cats = pd.read_csv(fp_cw, sep = "|")

In [23]:

def reshape_df_climate(
    df_input: pd.DataFrame,
    vec_x: np.ndarray,
    vec_y: np.ndarray,
    field_val: str = "value",
    field_x: str = "x",
    field_y: str = "y",
    flag_empty: int = flag_empty,
) -> pd.DataFrame:
    """
    Reshape df soils without having to melt and filter
    """
    arr = np.array(df_input)

    w = np.where(arr != flag_empty)
    
    # get indexing
    col_vals = arr[w[0], w[1]]
    col_x = vec_x[w[1]]
    col_y = vec_y[w[0]]
    
    df_out = pd.DataFrame(
        {
            field_x: col_x,
            field_y: col_y,
            field_val: col_vals
        }
    )
    
    return df_out



flag_empty = df_climates.iloc[0,0]
vec_x = np.array(df_climates.columns)
vec_y = np.array(df_climates.index)


dfc = reshape_df_climate(
    df_climates,
    vec_x,
    vec_y,
    flag_empty = flag_empty
)

crs_climates = "EPSG:4326"
gdf_climates_cut = dfc.copy()

In [36]:
#flag_empty = -3.4e+38
#np.where(np.array(df_climates.iloc[0:5, 0:5]) == )


In [47]:
"""
df_climates = df_climates.reset_index()#[-179.97083333333333].unique()
dfc = pd.melt(df_climates, ["y"]);
val = float(dfc["value"].iloc[0])
dfc = dfc[dfc["value"] != val].reset_index(drop = True)

# set some bounds
# http://bboxfinder.com/#-55.776573,-126.386719,33.431441,-33.398438
x_max = -34
x_min = -119
y_max = 33
y_min = -58
# filter out
dfc2 = dfc[
    (dfc["x"] <= x_max) & 
    (dfc["x"] >= x_min) & 
    (dfc["y"] <= y_max) & 
    (dfc["y"] >= y_min)
].reset_index(drop = True)

# create points with climates
gdf_climates = gpd.GeoDataFrame(
    dfc2, 
    geometry = gpd.points_from_xy(dfc2["x"], dfc2["y"])
)
gdf_climates.crs = "EPSG:4326"
gdf_climates_cut = gdf_climates.copy()
""";

In [34]:
# reload with filter
if False:
    

    dfc = pd.read_csv(os.path.join(dir_read, "kcc_coords_index.csv"), skiprows = inds_drop)
    
    if False:

        crs_climates = "EPSG:4326"

        gdf_climates = gpd.GeoDataFrame(
            dfc,
            geometry = gpd.points_from_xy(dfc["x"], dfc["y"], crs = crs_climates)
        )

In [41]:
import re
if False:
    regex_match = re.compile("kcc_by_country_(\D*).csv")
    dir_read = os.path.join(dir_data, "kcc_cells_merged_to_country")
    fls_read = sorted([x for x in os.listdir(dir_read) if regex_match.match(x) is not None])
    isos_avail = [regex_match.match(x).groups()[0] for x in fls_read]
    
    field_index = "index_right"
    inds_drop = []

    fl_tmp = "/Users/jsyme/Desktop/tmp.csv"

    #with open(fl_tmp, "w+") as fl:
    for i, fl in enumerate(fls_read):
        df_get = pd.read_csv(os.path.join(dir_read, fl))
        if field_index in df_get.columns:
            inds_drop += list(df_get[field_index])
            #inds_drop_cur = "\n".join([str(x) for x in inds_drop_cur])
            #fl.writelines(inds_drop_cur)

        print(f"{i} countries complete") if (i%25 == 0) else None
        
    
    crs_climates = "EPSG:4326"

    gdf_climates = gpd.GeoDataFrame(
        dfc,
        geometry = gpd.points_from_xy(dfc["x"], dfc["y"], crs = crs_climates)
    )

In [81]:
# get countries 
countries_keep = regions.all_isos#list(sa.model_attributes.dict_attributes.get("region").table["category_name"])
gdf_world = gpd.read_file(fp_countries)

# do some replacements to clean it up
field_formal_name = "FORMAL_EN"
dict_repl = {
    "French Republic": "FRA",
    "Kingdom of Norway": "NOR",
    "Republic of Kosovo": "XKX",
}
inds = gdf_world[gdf_world[field_formal_name].isin(dict_repl.keys())].index
gdf_world.loc[inds, field_en] = gdf_world.loc[inds][field_formal_name].replace(dict_repl)

# some replacements
field_en = "ISO_A3"
gdf_world_red = gdf_world[gdf_world[field_en].isin(countries_keep)]
gdf_lac = gdf_world[[field_en, "geometry"]]
gdf_lac_bounds = gdf_lac.bounds



In [85]:
#"FRA" in gdf_world[field_en]#[~gdf_lac[field_en].isin(isos_avail)][field_en].unique()
#[x for x in sorted(list(set(gdf_world["FORMAL_EN"].dropna()))) if ("fr" in x.lower())]

#gdf_world[~gdf_world[field_en].isin(isos_avail)][["FORMAL_EN"]].drop_duplicates()



,minx,miny,maxx,maxy
0,95.012706,-10.922621,140.977627,5.910102
1,99.645228,0.851370,119.278087,7.355780
2,-109.453725,-55.918504,-66.420806,-17.506588
3,-69.666492,-22.897258,-57.465661,-9.679821
4,-81.337558,-18.337746,-68.684252,-0.029093
...,...,...,...,...
246,-75.030832,18.395006,-75.002634,18.417263
247,-162.103627,5.869371,-162.057607,5.890245
248,-162.400177,6.430243,-162.385284,6.445136
249,-172.502553,-9.361749,-171.185658,-8.543227


In [66]:
def df_to_gdf(
    df_in: pd.DataFrame,
    crs: str = crs_climates,
    field_x: str = "x",
    field_y: str = "y",
) -> gpd.GeoDataFrame:
    
    vec_x = np.array(df_in[field_x])
    vec_y = np.array(df_in[field_y])
    
    gdf_out = gpd.GeoDataFrame(
        df_in, 
        geometry = [shapely.geometry.Point(*x) for x in zip(vec_x, vec_y)]
    )
    gdf_out.crs = crs
    
    return gdf_out
    
t0 = time.time()
gdf_points2 = df_to_gdf(gdf_points)
sf.get_time_elapsed(t0)

58.58

In [61]:
#sf.get_time_elapsed(t0)
import shapely
type(gdf_points2.iloc[0]["geometry"])

vec_x = np.array(gdf_points["x"])[0:100000]
vec_y = np.array(gdf_points["y"])[0:100000]

t0 = time.time()
xx = [shapely.geometry.Point(*x) for x in zip(vec_x, vec_y)]
print(sf.get_time_elapsed(t0))

t0 = time.time()
xx2 = gpd.points_from_xy(vec_x, vec_y, crs = crs_climates)
print(sf.get_time_elapsed(t0))


0.68
2.08


3

In [89]:
gdf_world_red = gdf_world[
    gdf_world[field_en].isin(countries_keep)
    & ~gdf_world[field_en].isin(isos_avail)
]
gdf_lac = gdf_world_red[[field_en, "geometry"]]
gdf_lac_bounds = gdf_lac.bounds



In [97]:

i = 0
sep = "---"*25
print("Copying climate gdf...\n\n")
gdf_climates_cut = gdf_climates.copy()


df_out = []
df_agg_out = []
t0 = time.time()
list_times = [t0]
n = len(gdf_lac_bounds)

while i < n:
    
    # country
    gdf_country = gdf_lac.iloc[i:(i + 1)]
    bounds = gdf_lac_bounds.iloc[i:(i + 1)]

    country = str(gdf_country[field_en].iloc[0])
    maxx = float(bounds["maxx"].iloc[0])
    minx = float(bounds["minx"].iloc[0])
    maxy = float(bounds["maxy"].iloc[0])
    miny = float(bounds["miny"].iloc[0])
    
    pos = i + 1
    print(f"Starting country {country}\n{sep} ({pos}/{n})\n")
    
    # get points
    print("Filtering points...\n")
    gdf_points = gdf_climates_cut[
        (gdf_climates_cut["x"] <= maxx) &
        (gdf_climates_cut["x"] >= minx) &
        (gdf_climates_cut["y"] <= maxy) &
        (gdf_climates_cut["y"] >= miny)
    ]
    
    if not isinstance(gdf_climates_cut, gpd.GeoDataFrame):
        print("Converting to GeoDataFrame...\n")
        gdf_points = gpd.GeoDataFrame(
            gdf_points,
            geometry = gpd.points_from_xy(gdf_points["x"], gdf_points["y"], crs = crs_climates)
        )
    
    print(f"Merging points within bbox x = ({minx}, {maxx}), y = ({miny}, {maxy}) to country...\n")
    gdf_join = gpd.sjoin(
        gdf_country, 
        gdf_points[["value", "geometry"]].rename(columns = {"value": "kcc"}),
        how = "inner"
    ).drop("geometry", axis = 1)
    
    
    print(f"Aggregating counts by climate...\n")
    gdf_agg = gdf_join[[field_en, "kcc"]].copy()
    gdf_agg["count"] = 1
    gdf_agg = (
        gdf_agg[[field_en, "kcc", "count"]]
        .groupby([field_en, "kcc"])
        .agg(
            {field_en: "first", "kcc": "first", "count": "sum"}
        )
        .reset_index(drop = True)
    )

    
    print("Appending output...")
    df_out.append(gdf_join)
    df_agg_out.append(gdf_agg)
    
    
    print("Dropping indices...")
    indices_to_drop = list(gdf_join["index_right"])
    gdf_climates_cut.drop(indices_to_drop, axis = 0, inplace = True)
    
    
    i += 1
    list_times.append(time.time())
    t_delta = np.round(list_times[i] - list_times[i - 1], 2)
    t_elapsed = np.round(list_times[i] - list_times[0])
    
    print(f"\n{sep}\n\nCountry {country} complete in {t_delta} seconds ({t_elapsed} seconds overall)\n\n{sep}\n\n\n")
    

Copying climate gdf...


Starting country FRA
--------------------------------------------------------------------------- (1/3)

Filtering points...

Merging points within bbox x = (-61.79784095039952, 55.85450280026407), y = (-21.370782159269197, 51.08754088370489) to country...

Aggregating counts by climate...

Appending output...
Dropping indices...

---------------------------------------------------------------------------

Country FRA complete in 58.79 seconds (59.0 seconds overall)

---------------------------------------------------------------------------



Starting country NOR
--------------------------------------------------------------------------- (2/3)

Filtering points...

Merging points within bbox x = (-9.117421027858086, 33.64039147178903), y = (-54.462497654443155, 80.77008698082955) to country...

Aggregating counts by climate...

Appending output...
Dropping indices...

---------------------------------------------------------------------------

Country NOR comp

# NOTE: CAN START HERE
- Only need to unzip and load `os.path.join(dir_data, "kcc_cells_merged_to_country", "kcc_coords_index.csv")`
- country-level values are stored in `os.path.join(dir_data, "kcc_cells_merged_to_country")` with name `f"kcc_by_country_{iso}.csv"`

In [83]:
pd.DataFrame(
    dfc
).to_csv(
    os.path.join(dir_data, "kcc_cells_merged_to_country", "kcc_coords_index.csv"),
    index = None,
    encoding = "UTF-8"
)

In [106]:
#df_out_full = pd.concat(df_out, axis = 0).reset_index(drop = True);
df_out_agg = pd.concat(df_agg_out, axis = 0).reset_index(drop = True);

# export output
#df_out_full.to_csv(sa.fp_csv_kcc_cells_merged_to_country, index = None, encoding = "UTF-8")
df_out_agg.to_csv(sa.fp_csv_kcc_cell_counts_by_country_kcc, index = None, encoding = "UTF-8")
"""
df_out_agg_original = pd.read_csv(sa.fp_csv_kcc_cell_counts_by_country_kcc)
df_out_agg_out  = (
    pd.concat(
        [
            df_out_agg_original,
            df_out_agg
        ],
        axis = 0
    )
    .sort_values(by = [field_en, "kcc"])
    .reset_index(drop = True)
)
""";

In [115]:
dir_exp = os.path.join(dir_data, "kcc_cells_merged_to_country")
os.makedirs(dir_exp, exist_ok = True)

for df in df_out:
    iso = str(df[field_en].iloc[0])
    fp_write = os.path.join(dir_exp, f"kcc_by_country_{iso}.csv")
    df.drop(field_en, axis = 1).to_csv(fp_write, index = None, encoding = "UTF-8")
    print(f"country '{iso}' complete.\n")

country 'FRA' complete.

country 'NOR' complete.

country 'XKX' complete.



In [32]:

#inds_drop
#pd.read_csv

In [116]:


##
##  BUILD FRACTIONS WET/DRY AND TEMPERATE/TROPICAL BY COUNTRY
##

df_out_agg["count"] = df_out_agg["count"].astype(int)

# get total counts
df_out_agg_totals = (
    sf.simple_df_agg(
        df_out_agg[[field_en, "count"]],
        [field_en],
        {"count": "sum"}
    )
    .rename(columns = {"count": "total_count"})
)
# 
df_climate_by_country = pd.merge(
    df_out_agg, 
    df_climate_cats[["code_num", "wet_dry_cat", "temperate_tropical_cat"]].rename(columns = {"code_num": "kcc"})
)

# break out by wet/dry and temperate/tropical
df_climate_by_country_temptrop = df_climate_by_country.drop(["wet_dry_cat"], axis = 1)
df_climate_by_country_wetdry = df_climate_by_country.drop(["temperate_tropical_cat"], axis = 1)
all_countries = sorted(list(set(df_climate_by_country[field_en])))

# loop over different splits
fields_loop = ["wet_dry_cat", "temperate_tropical_cat"]
dict_df_climate_aggs = {}


if True:
    for field in fields_loop:

        fields_grp = [field_en, field]
        fields_dat = ["count"]

        all_vals = sorted(list(set(df_climate_by_country[field])))
        df_tmp = df_climate_by_country.drop([x for x in fields_loop if (x != field)], axis = 1)
        dict_agg = dict(zip(fields_grp, ["first" for x in fields_grp]))
        dict_agg.update(dict(zip(fields_dat, ["sum" for x in fields_dat])))

        df_tmp = df_tmp[fields_grp + fields_dat].groupby(fields_grp).agg(dict_agg).reset_index(drop = True)
        df_expand = pd.DataFrame(itertools.product(all_countries, all_vals), columns = [field_en, field])
        df_expand = pd.merge(df_expand, df_out_agg_totals, how = "left").fillna(0)

        df_tmp = pd.merge(df_expand, df_tmp, how = "left").fillna(0)
        df_tmp["frac"] = np.array(df_tmp["count"])/np.array(df_tmp["total_count"])
        df_tmp.rename(columns = {field_en: "country"}, inplace = True)


        # get pivot
        df_piv = pd.pivot(
            df_tmp[["country", field, "frac"]],
            index = ["country"],
            columns = [field], 
            values = ["frac"]
        ).reset_index()
        df_piv.columns = df_piv.columns.to_flat_index()
        dict_rnm = dict([(x, x[1].replace(")", "").strip()) for x in df_piv.columns if (x[1] != "")])
        dict_rnm.update({("country", ""): "country"})
        df_piv.rename(columns = dict_rnm, inplace = True)

        dict_df_climate_aggs.update({field: df_piv})


In [118]:
##  TEMPERATE/TROPICAL SPLITS

varlist_tt = []
# ag temp/trop vars
varlist_tt += sa.model_attributes.build_variable_fields(model_afolu.modvar_agrc_frac_temperate)
varlist_tt += sa.model_attributes.build_variable_fields(model_afolu.modvar_agrc_frac_tropical)
# frst temp/trop vars
varlist_tt += sa.model_attributes.build_variable_fields(model_afolu.modvar_frst_frac_temperate_nutrient_rich)
varlist_tt += sa.model_attributes.build_variable_fields(model_afolu.modvar_frst_frac_temperate_nutrient_poor)
varlist_tt += sa.model_attributes.build_variable_fields(model_afolu.modvar_frst_frac_tropical)
# lndu temp/trop vars
varlist_tt += sa.model_attributes.build_variable_fields(model_afolu.modvar_lndu_frac_temperate)
varlist_tt += sa.model_attributes.build_variable_fields(model_afolu.modvar_lndu_frac_tropical)


##  WET/DRY SPLITS

varlist_wd = []
# ag temp/trop vars
varlist_wd += sa.model_attributes.build_variable_fields(model_afolu.modvar_agrc_frac_wet)
varlist_wd += sa.model_attributes.build_variable_fields(model_afolu.modvar_agrc_frac_dry)
# lndu temp/trop vars
varlist_wd += sa.model_attributes.build_variable_fields(model_afolu.modvar_lndu_frac_wet)
varlist_wd += sa.model_attributes.build_variable_fields(model_afolu.modvar_lndu_frac_dry)



In [119]:

df_years = pd.DataFrame({time_periods.field_year: range(2010, time_periods.year_max + 1)})

df_ret = dict_df_climate_aggs[field][["country"]].copy()

for field in dict_df_climate_aggs.keys():
    
    loop = varlist_tt if (field == "temperate_tropical_cat") else varlist_wd
    for var in loop:
        if field == "temperate_tropical_cat":
            fld = "temperate" if ("temperate" in var) else ("tropical")
        else:
            fld = "wet" if ("wet" in var) else "dry"
        div = 2 if (("nutrient_poor" in var) or ("nutrient_rich" in var)) else 1
        df_merge = dict_df_climate_aggs[field][["country", fld]].copy().rename(columns = {fld: var})
        df_ret = pd.merge(df_ret, df_merge)
        df_ret[var] = np.array(df_ret[var])/div

# merge in time period
df_ret = sf.explode_merge(
    df_years,
    df_ret
)
df_ret = time_periods.years_to_tps(df_ret)
df_ret.rename(columns = {"country": regions.field_iso}, inplace = True)
fields_sort = [regions.field_iso, time_periods.field_time_period, time_periods.field_year]

df_ret = (
    df_ret[fields_sort + sorted([x for x in df_ret.columns if (x not in fields_sort)])]
    .sort_values(by = fields_sort)
    .reset_index(drop = True)
)

if False:

    df_ret.to_csv(
        sa.fp_csv_climate_fields_by_country_simple, index = None, encoding = "UTF-8"
    )

In [128]:
#[[fields_sort] + sorted([x for x in df_ret.columns if (x not in fields_sort)])]
df_ret_original = pd.read_csv(sa.fp_csv_climate_fields_by_country_simple)
df_ret_out  = (
    pd.concat(
        [
            df_ret_original,
            df_ret
        ],
        axis = 0
    )
    .sort_values(by = [regions.field_iso, time_periods.field_year])
    .reset_index(drop = True)
)

In [123]:


for var_name in varlist_soc:
    df_agg_all_fields[var_name] = np.array(df_agg_all_fields[field_cp])

df_agg_all_fields.drop(field_cp, axis = 1, inplace = True)
df_agg_all_fields.rename(columns = {field_en: field_country}, inplace = True)
    




,country,time_period,frac_agrc_bevs_and_spices_cl1_temperate,frac_agrc_bevs_and_spices_cl1_tropical,frac_agrc_bevs_and_spices_cl2_dry,frac_agrc_bevs_and_spices_cl2_wet,frac_agrc_cereals_cl1_temperate,frac_agrc_cereals_cl1_tropical,frac_agrc_cereals_cl2_dry,frac_agrc_cereals_cl2_wet,...,frac_frst_secondary_cl1_temperate_nutrient_rich,frac_frst_secondary_cl1_tropical,frac_lndu_grasslands_cl1_temperate,frac_lndu_grasslands_cl1_tropical,frac_lndu_grasslands_cl2_dry,frac_lndu_grasslands_cl2_wet,frac_lndu_other_cl2_dry,frac_lndu_other_cl2_wet,frac_lndu_settlements_cl2_dry,frac_lndu_settlements_cl2_wet
0,ABW,0,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.500000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
1,ABW,1,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.500000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
2,ABW,2,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.500000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
3,ABW,3,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.500000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
4,ABW,4,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.500000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7627,ZWE,31,0.989726,0.010274,0.989726,0.010274,0.989726,0.010274,0.989726,0.010274,...,0.494863,0.010274,0.989726,0.010274,0.989726,0.010274,0.989726,0.010274,0.989726,0.010274
7628,ZWE,32,0.989726,0.010274,0.989726,0.010274,0.989726,0.010274,0.989726,0.010274,...,0.494863,0.010274,0.989726,0.010274,0.989726,0.010274,0.989726,0.010274,0.989726,0.010274
7629,ZWE,33,0.989726,0.010274,0.989726,0.010274,0.989726,0.010274,0.989726,0.010274,...,0.494863,0.010274,0.989726,0.010274,0.989726,0.010274,0.989726,0.010274,0.989726,0.010274
7630,ZWE,34,0.989726,0.010274,0.989726,0.010274,0.989726,0.010274,0.989726,0.010274,...,0.494863,0.010274,0.989726,0.010274,0.989726,0.010274,0.989726,0.010274,0.989726,0.010274
